In [1]:
# coding: utf-8

# In[ ]:


# Title: HOG And MSER Classifiers On CBS's Data
# Author: Florian Koenigstorfer
# Date of last edit: 22/03/2018

### File Description
# This file is intended to test how well a SVM with HOGs and MSER input performs
# on recognising satelites in aerial images.

# Import Libraries
import glob
import os
import cv2       # Currently version 3.1
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.externals import joblib

In [2]:
import cv2
from os import listdir
import numpy as np

path_negatives="./tim_samples/Negatives/"

path_positives="./tim_samples/Positives/"

img_rows =224
img_cols = 224


#first 100
imagesListNegatives = listdir(path_negatives)

#second 100
imagesListPositives = listdir(path_positives)


labels = np.zeros((len(imagesListNegatives)+len(imagesListPositives)))
labels[len(imagesListPositives):len(imagesListNegatives)+len(imagesListPositives)] = 1


ListImgs = []
for i in imagesListNegatives:
    img = cv2.imread(path_negatives+str(i))
    img = cv2.resize(img,(224,224),interpolation = cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    ListImgs.append(img)

for i in imagesListPositives:
    img = cv2.imread(path_positives+str(i))
    img = cv2.resize(img,(224,224),interpolation = cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    ListImgs.append(img)

NListIm = np.asarray(ListImgs)
#NListIm =NListIm.reshape(NListIm.shape[0],3,img_rows,img_cols) # preparing for tensor...theano version, use above with tf
#We have to extract features from everything then split in training and testing



In [ ]:
### Function (3) - Retrieving HOGs features 

## Input:
#     Image from which we want to retrieve the HOG features
#         (Needs to be 2d image. In udemy course, the image
#          was gray-scaled. But Pieter suggested deriving HOG
#          features from individual R[or G or B] layers).
#     Cell size (cell_size)    ... tuple of 2 inputs (hxw; in pixels)
#     Block size (block_size)  ... tuple of 2 inputs (hxw; in cells)
#     Number of Bins (nbins)   ... scalar
#
## Output:
#     HOG features
#     Gradients
#
## Requirements:
#     cv2/OpenCV (Library)
#
# Source: Udemy course OpenCV

def derive_HOG_features(image, cell_size, block_size, nbins):
    # HOGs
    # image = image
    gray = image
    
    # h x w in pixels
    # cell_size = (1, 1)
    
    # h x w in cells
    # block_size = (2, 2) 
    
    # number of orientation bins
    # nbins = 9
    
    # Using OpenCV's HOG Descriptor
    # winSize is the size of the image cropped to a multiple of the cell size
    hog = cv2.HOGDescriptor(_winSize=(gray.shape[1] // cell_size[1] * cell_size[1],
                                      gray.shape[0] // cell_size[0] * cell_size[0]),
                            _blockSize=(block_size[1] * cell_size[1],
                                        block_size[0] * cell_size[0]),
                            _blockStride=(cell_size[1], cell_size[0]),
                            _cellSize=(cell_size[1], cell_size[0]),
                            _nbins=nbins)
    
    # Create numpy array shape which we use to create hog_feats
    n_cells = (gray.shape[0] // cell_size[0], gray.shape[1] // cell_size[1])
    
    # We index blocks by rows first.
    # hog_feats now contains the gradient amplitudes for each direction,
    # for each cell of its group for each group. Indexing is by rows then columns.
    hog_feats = hog.compute(gray).reshape(n_cells[1] - block_size[1] + 1,
                                          n_cells[0] - block_size[0] + 1,
                                          block_size[0], block_size[1], nbins).transpose((1, 0, 2, 3, 4))
    
    # Create our gradients array with nbin dimensions to store gradient orientations
    gradients = np.zeros((n_cells[0], n_cells[1], nbins))
    
    # Create array of dimensions
    cell_count = np.full((n_cells[0], n_cells[1], 1), 0, dtype=int)
    
    # Block Normalization
    for off_y in range(block_size[0]):
        for off_x in range(block_size[1]):
            gradients[off_y:n_cells[0] - block_size[0] + off_y + 1,
                      off_x:n_cells[1] - block_size[1] + off_x + 1] += \
            hog_feats[:, :, off_y, off_x, :]
            cell_count[off_y:n_cells[0] - block_size[0] + off_y + 1,
                       off_x:n_cells[1] - block_size[1] + off_x + 1] += 1
    # Average gradients
    gradients /= cell_count
    
    # Printing
    # print("Iteration done.")
    
    return(hog_feats, gradients)

### Function (4) - Importing names of all files with a given ending from a given folder
## Input:
#     Name of the folder location
#     Ending of the type of image I am interested in
#
## Output:
#     List of names, containing the images I am interested in.
#
## Requirements:
#     os (Library)
#     glog (Library)

def load_names_from_folder(folder,ending):
    # Saving current WD and changing WD for importing images
    original_working_directory = os.getcwd()
    os.chdir(folder)
    
    # Getting filenames with given ending & Setting up vector
    names_of_images = glob.glob(ending)
   
    # Setting working directory back to original
    os.chdir(original_working_directory)
    
    # Returning images
    return(names_of_images)

# Import Data

# Defining Directories
# Directory Positives Server 2 (Ubuntu): '/home/biss/Documents/Florian/Positives'
Pos_dir = '/home/biss/Documents/Florian/Positives'
# Directory Negaitives Server 2 (Ubuntu): '/home/biss/Documents/Florian/Negatives'
Neg_dir = '/home/biss/Documents/Florian/Negatives'

ending = '*.png'

# Importing images
positive_images = load_images_from_folder(Pos_dir,ending)
positive_images_names = load_names_from_folder(Pos_dir,ending)
negative_images = load_images_from_folder(Neg_dir,ending)
negative_images_names = load_names_from_folder(Neg_dir,ending)


### What is the shape of the largest image?
# Initializing lengths of images
pos_im_length = []
neg_im_length = []

# Initializing widths of images
pos_im_width = []
neg_im_width = []

# Extracting size of positive images
for i in range(0,(len(positive_images))):
    temp = positive_images[i].shape
    pos_im_length.append(temp[0])
    pos_im_width.append(temp[1])

# Extracting size of negative images
for i in range(0,(len(negative_images))):
    temp = negative_images[i].shape
    neg_im_length.append(temp[0])
    neg_im_width.append(temp[1])

# Finding maximum shape parameters
max_length_pos = max(pos_im_length)
max_length_neg = max(neg_im_length)
max_length = max(max_length_pos,max_length_neg)

max_width_pos = max(pos_im_length)
max_width_neg = max(pos_im_length)
max_width = max(max_width_pos,max_width_neg)

desired_size_of_images = max(max_length,max_width)

# Reshaping positive images
positive_images_reshaped = []
for i in positive_images:
    temp = re_shapping(i,desired_size_of_images)
    positive_images_reshaped.append(temp)

# Reshaping negative images
negative_images_reshaped = []
for i in negative_images:
    temp = re_shapping(i,desired_size_of_images)
    negative_images_reshaped.append(temp)

### HOGs features
# Inputs
cell_size= (1,1)
block_size = (2,2)
nbins = 9

# Positives
pos_grad_R= []
pos_hog_feat_R =[]
pos_grad_G = []
pos_hog_feat_G =[]
pos_grad_B =[]
pos_hog_feat_B =[]
pos_grad_Gray= []
pos_hog_feat_Gray =[]
for i in positive_images_reshaped:
    # R
    temp = i[:,:,0]
    temp = derive_HOG_features(temp, cell_size, block_size, nbins)
    pos_hog_feat_R.append(temp[0])
    pos_grad_R.append(temp[1])
    # G
    temp = i[:,:,1]
    temp = derive_HOG_features(temp, cell_size, block_size, nbins)
    pos_hog_feat_G.append(temp[0])
    pos_grad_G.append(temp[1])
    # B
    temp = i[:,:,2]
    temp = derive_HOG_features(temp, cell_size, block_size, nbins)
    pos_hog_feat_B.append(temp[0])
    pos_grad_B.append(temp[1])
    # Gray
    temp = cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)
    temp = derive_HOG_features(temp, cell_size, block_size, nbins)
    pos_hog_feat_Gray.append(temp[0])
    pos_grad_Gray.append(temp[1])

# Negatives
neg_grad_R= []
neg_hog_feat_R =[]
neg_grad_G = []
neg_hog_feat_G =[]
neg_grad_B =[]
neg_hog_feat_B =[]
neg_grad_Gray= []
neg_hog_feat_Gray =[]
for i in negative_images_reshaped:
    # R
    temp = i[:,:,0]
    temp = derive_HOG_features(temp, cell_size, block_size, nbins)
    neg_hog_feat_R.append(temp[0])
    neg_grad_R.append(temp[1])
    # G
    temp = i[:,:,1]
    temp = derive_HOG_features(temp, cell_size, block_size, nbins)
    neg_hog_feat_G.append(temp[0])
    neg_grad_G.append(temp[1])
    # B
    temp = i[:,:,2]
    temp = derive_HOG_features(temp, cell_size, block_size, nbins)
    neg_hog_feat_B.append(temp[0])
    neg_grad_B.append(temp[1])
    # Gray
    temp = cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)
    temp = derive_HOG_features(temp, cell_size, block_size, nbins)
    neg_hog_feat_Gray.append(temp[0])
    neg_grad_Gray.append(temp[1])


### MSER blob detection
# Source: Mastering OpenCV course on Udemy

# Defining detector
detector = cv2.MSER_create()

# Detecting blobs in positive images
positive_images_blobs = []
for i in positive_images_reshaped:
    temp = detector.detect(i)
    positive_images_blobs.append(temp)


# Detecting blobs in negative images
negative_images_blobs = []
for i in negative_images_reshaped:
    temp = detector.detect(i)
    negative_images_blobs.append(temp)

# Getting data into the correct format
# Source: http://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_ml/py_svm/py_svm_opencv/py_svm_opencv.html#svm-opencv

# Defining training and testing set
neg_grad_R = np.float32(neg_grad_R)
neg_hog_feat_R = np.float32(neg_hog_feat_R)
neg_grad_G = np.float32(neg_grad_G)
neg_hog_feat_G = np.float32(neg_hog_feat_G)
neg_grad_B = np.float32(neg_grad_B)
neg_hog_feat_B = np.float32(neg_hog_feat_B)
neg_grad_Gray = np.float32(neg_grad_Gray)
neg_hog_feat_Gray = np.float32(neg_hog_feat_Gray)
# negative_images_blobs = np.float32(negative_images_blobs)

pos_grad_R = np.float32(pos_grad_R)
pos_hog_feat_R = np.float32(pos_hog_feat_R)
pos_grad_G = np.float32(pos_grad_G)
pos_hog_feat_G = np.float32(pos_hog_feat_G)
pos_grad_B = np.float32(pos_grad_B)
pos_hog_feat_B = np.float32(pos_hog_feat_B)
pos_grad_Gray = np.float32(pos_grad_Gray)
pos_hog_feat_Gray = np.float32(pos_hog_feat_Gray)

### Printing HOGs gradients to file

### Info about the dimensions of the list/array
# nd.array [1][2][3][4]
# length   [11][248][248]
# pos_grad_R[0][0][0] = [ 0.  0.  0.  0.  0.  0.  0.  0.  0.], length = 9

### Positives
# Setting parameters of inner loops
array_shape = pos_grad_R[0].shape
MSER_list_shape = len(positive_images_blobs)
feature_array_shape = pos_hog_feat_R[0].shape

# This will be the file to be printed
one_dimensional_list_pos = []

# Convert array into one dimensional array
for dim1 in range(len(pos_grad_R)):
    
    temp_dim2 = []
    temp_dim3 = []
    
        
    for dim2 in range(array_shape[0]):
        for dim3 in range(array_shape[1]):
            
            ## HOG gradients
            temp_dim3 = pos_grad_R[dim1][dim2][dim3].tolist() 
            temp_dim3 = temp_dim3 + pos_grad_G[dim1][dim2][dim3].tolist()
            temp_dim3 = temp_dim3 + pos_grad_B[dim1][dim2][dim3].tolist()
            temp_dim3 = temp_dim3 + pos_grad_Gray[dim1][dim2][dim3].tolist()
            
            ## HOGs features
            temp_dim4 = []
            for dim4 in range(feature_array_shape[2]):
                temp_dim5 = []
                for dim5 in range(feature_array_shape[3]):
                    if(dim2==247):
                        break
                    elif(dim3==247):
                        break
                    elif(dim4==3):
                        break
                    else:
                        temp_dim5.append(pos_hog_feat_R[dim1][dim2][dim3][dim4].tolist())
                        temp_dim5.append(pos_hog_feat_G[dim1][dim2][dim3][dim4].tolist())
                        temp_dim5.append(pos_hog_feat_B[dim1][dim2][dim3][dim4].tolist())
                        temp_dim5.append(pos_hog_feat_Gray[dim1][dim2][dim3][dim4].tolist())
                if(dim2==247):
                    break
                elif(dim3==247):
                    break
                elif(dim4==3):
                    break
                else:
                    temp_dim4.append(temp_dim5)
            
            temp_dim2 = temp_dim2 + temp_dim3 + temp_dim4
       
    # Appending the result to the array
    one_dimensional_list_pos.append(temp_dim2)
    
# Printing
thefile = open('HOGs_positives.txt', 'w')
for item in one_dimensional_list_pos:
  thefile.write("%s\n" % item)



### Negatives
# Setting parameters of inner loops
array_shape = neg_grad_R[0].shape
MSER_list_shape = len(negative_images_blobs)
feature_array_shape = neg_hog_feat_R[0].shape

# This will be the file to be printed
one_dimensional_list_neg = []

# Convert array into one dimensional array
for dim1 in range(len(neg_grad_R)):
    
    temp_dim2 = []
    temp_dim3 = []
    
    for dim2 in range(array_shape[0]):
        for dim3 in range(array_shape[1]):
             ## HOG gradients
            temp_dim3 = pos_grad_R[dim1][dim2][dim3].tolist() 
            temp_dim3 = temp_dim3 + neg_grad_G[dim1][dim2][dim3].tolist()
            temp_dim3 = temp_dim3 + neg_grad_B[dim1][dim2][dim3].tolist()
            temp_dim3 = temp_dim3 + neg_grad_Gray[dim1][dim2][dim3].tolist()
            
            ## HOGs features
            temp_dim4 = []
            for dim4 in range(feature_array_shape[2]):
                temp_dim5 = []
                for dim5 in range(feature_array_shape[3]):
                    if(dim2==247):
                        break
                    elif(dim3==247):
                        break
                    elif(dim4==3):
                        break
                    else:
                        temp_dim5.append(neg_hog_feat_R[dim1][dim2][dim3][dim4].tolist())
                        temp_dim5.append(neg_hog_feat_G[dim1][dim2][dim3][dim4].tolist())
                        temp_dim5.append(neg_hog_feat_B[dim1][dim2][dim3][dim4].tolist())
                        temp_dim5.append(neg_hog_feat_Gray[dim1][dim2][dim3][dim4].tolist())
                if(dim2==247):
                    break
                elif(dim3==247):
                    break
                elif(dim4==3):
                    break
                else:
                    temp_dim4.append(temp_dim5)
            
            temp_dim2 = temp_dim2 + temp_dim3 + temp_dim4
    
    # Appending the result to the array
    one_dimensional_list_neg.append(temp_dim2)
    
# Printing
thefile = open('HOGs_negatives.txt', 'w')
for item in one_dimensional_list_neg:
  thefile.write("%s\n" % item)

### Importing positives from file
one_dimensional_list_pos = []
with open('positives.txt') as inputfile:
    for line in inputfile:
        one_dimensional_list_pos.append(line.strip().split(','))

for i in range(len(one_dimensional_list_neg)):
    one_dimensional_list_pos[i][0] = one_dimensional_list_pos[i][0][1:]
    one_dimensional_list_pos[i][-1] = one_dimensional_list_pos[i][-1][:-1]
    one_dimensional_list_pos[i] = [float(j) for j in one_dimensional_list_pos[i]]




### Importing negatives from file
one_dimensional_list_neg = []
with open('negatives.txt') as inputfile:
    for line in inputfile:
        one_dimensional_list_neg.append(line.strip().split(','))

for i in range(len(one_dimensional_list_pos)):
    one_dimensional_list_neg[i][0] = one_dimensional_list_neg[i][0][1:]
    one_dimensional_list_neg[i][-1] = one_dimensional_list_neg[i][-1][:-1]
    one_dimensional_list_neg[i] = [float(j) for j in one_dimensional_list_neg[i]]

### SVM using sklearn
# Source: http://scikit-learn.org/stable/modules/svm.html
X = np.asarray(one_dimensional_list_pos + one_dimensional_list_neg)
Y = np.asarray(np.ones(len(one_dimensional_list_pos)).tolist() + np.zeros(len(one_dimensional_list_neg)).tolist())

clf = svm.SVC()
clf.fit(X, Y)

# Saving model weights
joblib.dump(clf, 'filename.pkl') 

# Loading model weights
# clf = joblib.load('filename.pkl') 